### import functions

In [1]:
from sklearn.model_selection  import cross_val_score, train_test_split, cross_validate, cross_val_score,KFold,RandomizedSearchCV,RepeatedKFold, GridSearchCV,StratifiedKFold
from sklearn import svm,tree, neighbors,linear_model, preprocessing
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, r2_score
from sklearn import metrics
import xgboost as xgb
from lightgbm import LGBMClassifier
import lightgbm as lgb
import numpy as np
import pandas as pd 
from sklearn.model_selection  import cross_val_score, train_test_split, cross_validate, cross_val_score,KFold,RandomizedSearchCV,RepeatedKFold, GridSearchCV,StratifiedKFold

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import missingno as msno 

### load in data

In [2]:
app_train = pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/application_train.csv')
app_test = pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/application_test.csv')
bureau= pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/bureau.csv')
b_balance= pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/bureau_balance.csv')
c_balance= pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/credit_card_balance.csv')
install= pd.read_csv('C:/Users/yidan/Desktop/fall/PA/Project/installments_payments.csv')

### categorical -> numerical

In [3]:
#categorical variable with less than 
# 3 unique values, encode to 0,1,2
le = LabelEncoder()
le_count = 0

In [4]:
# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [5]:
#categorical variable with more than 
# 3 unique values, convert to dummy variables
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [6]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

There are 9274 anomalies in the test data out of 48744 entries


### Label Y

In [7]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


### impute missing values

In [8]:
#filling in the missing values (imputation) and 
#normalizing the range of the features (feature scaling)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

In [9]:
# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

In [10]:
# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 239)
Testing data shape:  (48744, 239)


### PCA

In [11]:
pca = PCA(n_components=98)
pca.fit(train)
pca_train = pca.transform(train)
pca.fit(test)
pca_test = pca.transform(test)

In [12]:
print('Training data shape: ', pca_train.shape)
print('Testing data shape: ', pca_test.shape)

Training data shape:  (307511, 98)
Testing data shape:  (48744, 98)


### split train test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    pca_train, train_labels, train_size=0.8)

### Logistic Reg

In [14]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
k_fold = StratifiedKFold(n_splits=2)
clf_log = linear_model.LogisticRegression(solver='liblinear')

param_range = [0.01,0.1, 1, 10, 100]
params= {'C': param_range, 'penalty': ['l1','l2']}
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(clf_log, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)
clf_log = grid.best_estimator_

print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_log.get_params()[key]))
print('Logistic Regression R2: %f'%clf_log.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))

best parameters:
C: 0.01
penalty: l1
Logistic Regression R2: 0.919711
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56569
           1       0.40      0.00      0.00      4934

    accuracy                           0.92     61503
   macro avg       0.66      0.50      0.48     61503
weighted avg       0.88      0.92      0.88     61503



### Random Forest

In [15]:
#Random Forest
from sklearn import ensemble
clf_rf=ensemble.RandomForestClassifier()

k_fold=KFold(n_splits=2)
params= {'n_estimators': range(1,5), 'max_depth': range(1,5)}
scoring_fnc = make_scorer(accuracy_score)
rf_grid = GridSearchCV(clf_rf, param_grid=params,scoring=scoring_fnc,cv=k_fold)
rf_grid = rf_grid.fit(X_train, y_train)
clf_rand= rf_grid.best_estimator_

print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_rf.get_params()[key]))
print('Random Forest R2: %f'%clf_rand.score(X_test, y_test))
print(classification_report(y_test,rf_grid.predict(X_test)))

best parameters:
n_estimators: 100
max_depth: None
Random Forest R2: 0.919776
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56569
           1       0.00      0.00      0.00      4934

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.85      0.92      0.88     61503



### XGBoost

In [24]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=10)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)
print("XGBoost's prediction accuracy is: %3.4f" % (acc_xgb))
print(classification_report(y_test,xgb.predict(X_test)))

[19:07:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost's prediction accuracy is: 0.9209
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56638
           1       0.60      0.00      0.00      4865

    accuracy                           0.92     61503
   macro avg       0.76      0.50      0.48     61503
weighted avg       0.90      0.92      0.88     61503



### LightGBM

In [25]:
rmodel = lgb.LGBMClassifier()

k_fold=KFold(n_splits=2)
scoring_fnc = make_scorer(accuracy_score)
params= {'n_estimators': range(1,5), 'max_depth': range(1,5), 'learning_rate': [0.01, 0.05, 0.1]}
grid = GridSearchCV(rmodel, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = rmodel.fit(X_train, y_train)
preds = rmodel.predict(X_test)

acc_lgb = (preds == y_test).sum().astype(float) / len(preds)
print("lgb's prediction accuracy is: %3.4f" % (acc_lgb))
print(classification_report(y_test,grid.predict(X_test)))

lgb's prediction accuracy is: 0.9206
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56638
           1       0.41      0.01      0.01      4865

    accuracy                           0.92     61503
   macro avg       0.67      0.50      0.49     61503
weighted avg       0.88      0.92      0.88     61503

